<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BANK_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask Flask-CORS Flask-SQLAlchemy pyngrok colab-env -q
!pip install Flask-Login -q

In [4]:
from flask import Flask, jsonify, request, send_from_directory, render_template, redirect, url_for, flash
from flask_cors import CORS
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
import os
import random # For retry jitter
import time # For retry backoff
from pyngrok import ngrok, conf
from sqlalchemy.orm import relationship

# --- LLM Integration Imports ---
from openai import OpenAI, APIStatusError # Import OpenAI client and specific exception

# --- Colab Environment (Optional, for setting env vars in Colab) ---
import colab_env
from google.colab import userdata # Added for secure API key retrieval in Colab

# --- Flask App Initialization ---
app = Flask(__name__, static_folder='/content/static', template_folder='templates')
CORS(app)

# --- Secret Key for Sessions and Flask-Login ---
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY', 'your_super_secret_key_change_this_in_production_2025')

# --- Database Configuration ---
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/site.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# --- Flask-Login Setup ---
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

# --- Database Models ---
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    account = relationship('Account', backref='owner', uselist=False)

    def __repr__(self):
        return f'<User {self.username}>'

class Account(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), unique=True, nullable=False)
    balance = db.Column(db.Float, nullable=False, default=0.0)

    def __repr__(self):
        return f'<Account User:{self.user_id} Balance:{self.balance:.2f}>'

# --- Flask-Login user loader callback ---
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# --- Global variable for ngrok public URL ---
public_url = "http://127.0.0.1:5000"

## BEGIN deepseek API
# Retrieve API key from Colab userdata. This is specific to Google Colab.
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
# The 'model' variable is often used for clarity, but the actual model name
# is passed directly to the client.chat.completions.create call.
# model="deepseek-reasoner"
client = None # Initialize client globally
if deepseek_api_key:
    try:
        # Initialize the OpenAI client configured for DeepSeek's API
        client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
        print("DeepSeek API client initialized successfully using userdata.")
    except Exception as e:
        print(f"ERROR: Failed to initialize DeepSeek API client: {e}")
        client = None # Ensure client is None if initialization fails
else:
    print("WARNING: DEEPSEEK_API_KEY not found in Colab userdata. LLM features will not work.")
## END deepseek API


# --- LLM Call with Retry Logic (Adapted for OpenAI client) ---
def call_llm_with_retry(prompt, max_retries=3):
    """
    Calls the LLM (DeepSeek via OpenAI client) with retry logic.
    """
    if client is None:
        raise ValueError("DeepSeek API client is not initialized. Cannot call LLM.")

    # Define system instruction for the LLM
    system_instruction = (
        "You are a helpful and friendly AI assistant for a SuperApp that includes banking, "
        "shopping, and travel features. Your primary goal is to assist the user within these domains. "
        "Remember, this is FIRST a BANKING application. "
        "You are also being developed for flight planning optimization, drawing on principles "
        "from Newton, Galileo, Einstein, and Hinton. Keep responses concise and relevant."
    )

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": prompt},
    ]

    for i in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="deepseek-reasoner", # Specify the DeepSeek model
                max_tokens=2024,
                temperature=0.7,
                top_p=1,
                messages=messages,
                stream=False
            )
            if response and response.choices and response.choices[0] and response.choices[0].message:
                return response.choices[0].message.content
            else:
                return "LLM returned an empty or unexpected response structure."

        except APIStatusError as e:
            print(f"DeepSeek APIStatusError ({e.status_code}): {e.response} (Attempt {i+1}/{max_retries})")
            if e.status_code == 429 or e.status_code >= 500:
                if i < max_retries - 1:
                    delay = (2 ** i) + random.random()
                    print(f"Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                else:
                    print(f"Max retries reached for APIStatusError.")
                    raise e
            else:
                raise e
        except Exception as e:
            print(f"An unexpected error occurred during LLM call (Attempt {i+1}/{max_retries}): {e}")
            raise e


# --- Routes ---

@app.route('/')
def index():
    return render_template('index.html', public_url=public_url)

@app.route('/favicon.ico')
def favicon():
    return send_from_directory(app.static_folder, 'favicon.ico', mimetype='image/vnd.microsoft.icon')

# --- API Endpoints for Banking SuperApp ---

@app.route('/api/status', methods=['GET'])
@login_required
def get_status():
    user_account = current_user.account
    if not user_account:
        return jsonify(isLoggedIn=True, username=current_user.username, balance=None, message="No account found for user"), 404

    return jsonify(
        isLoggedIn=True,
        username=current_user.username,
        balance=user_account.balance,
        message="User status and balance retrieved."
    ), 200

@app.route('/api/deposit', methods=['POST'])
@login_required
def deposit():
    data = request.json
    amount = data.get('amount')

    if not amount or not isinstance(amount, (int, float)) or amount <= 0:
        return jsonify(success=False, message="Invalid deposit amount. Must be a positive number."), 400

    user_account = current_user.account
    if not user_account:
        return jsonify(success=False, message="No account found for user."), 404

    try:
        user_account.balance += float(amount)
        db.session.commit()
        return jsonify(success=True, newBalance=user_account.balance, message=f"Deposited {amount:.2f}. New balance: {user_account.balance:.2f}"), 200
    except Exception as e:
        db.session.rollback()
        return jsonify(success=False, message=f"Error processing deposit: {str(e)}"), 500

@app.route('/api/withdraw', methods=['POST'])
@login_required
def withdraw():
    data = request.json
    amount = data.get('amount')

    if not amount or not isinstance(amount, (int, float)) or amount <= 0:
        return jsonify(success=False, message="Invalid withdrawal amount. Must be a positive number."), 400

    user_account = current_user.account
    if not user_account:
        return jsonify(success=False, message="No account found for user."), 404

    if user_account.balance < amount:
        return jsonify(success=False, message="Insufficient funds."), 400

    try:
        user_account.balance -= float(amount)
        db.session.commit()
        return jsonify(success=True, newBalance=user_account.balance, message=f"Withdrew {amount:.2f}. New balance: {user_account.balance:.2f}"), 200
    except Exception as e:
        db.session.rollback()
        return jsonify(success=False, message=f"Error processing withdrawal: {str(e)}"), 500

@app.route('/api/agent_chat', methods=['POST'])
@login_required
def agent_chat():
    """
    Integrates with DeepSeek LLM via OpenAI client to provide AI agent responses.
    """
    data = request.json
    user_message = data.get('message', '')

    if not user_message:
        return jsonify(success=False, message="No message provided for agent chat."), 400

    if client is None: # Check if LLM client is initialized
        return jsonify(success=False, message="AI Agent is currently unavailable (DeepSeek API client not configured)."), 503

    try:
        agent_response = call_llm_with_retry(prompt=user_message)

        return jsonify(success=True, response=agent_response), 200
    except ValueError as e: # Catch if API key is not set (from call_llm_with_retry)
        return jsonify(success=False, message=f"AI Agent configuration error: {str(e)}"), 500
    except APIStatusError as e: # Catch specific OpenAI API errors
        print(f"APIStatusError during agent chat: {e}")
        return jsonify(success=False, message=f"Error from AI Agent service ({e.status_code}): {e.message}"), 500
    except Exception as e: # Catch any other unexpected errors
        print(f"Unhandled error during agent chat: {e}")
        return jsonify(success=False, message=f"An unexpected error occurred with the AI Agent: {str(e)}. Please try again."), 500


# --- Authentication Routes (Flask-Login) ---

@app.route('/register', methods=['POST'])
def register():
    if current_user.is_authenticated:
        return jsonify(message='Already logged in. Cannot register new account while logged in.', success=False), 400

    username = request.form.get('username')
    email = request.form.get('email')
    password = request.form.get('password')

    if not username or not email or not password:
        return jsonify(message='Missing username, email, or password.', success=False), 400

    if User.query.filter_by(username=username).first():
        return jsonify(message='Username already exists.', success=False), 409
    if User.query.filter_by(email=email).first():
        return jsonify(message='Email already exists.', success=False), 409

    try:
        new_user = User(username=username, email=email)
        db.session.add(new_user)
        db.session.commit()

        new_account = Account(user_id=new_user.id, balance=0.0)
        db.session.add(new_account)
        db.session.commit()

        return jsonify(message='Registration successful! You can now log in.', success=True), 201
    except Exception as e:
        db.session.rollback()
        return jsonify(message=f'Registration failed: {str(e)}', success=False), 500


@app.route('/login', methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return jsonify(message='Already logged in!', success=True), 200

    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        user = User.query.filter_by(username=username).first()

        if user:
            login_user(user)
            return jsonify(message=f'Logged in as {user.username}!', success=True), 200
        return jsonify(message='Invalid username or password', success=False), 401

    return jsonify(message='Please use POST request to login.', success=False), 405

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return jsonify(message='You have been logged out.', success=True), 200

# --- Application Startup ---
if __name__ == '__main__':
    with app.app_context():
        db_path = app.config['SQLALCHEMY_DATABASE_URI'].replace('sqlite:///', '')
        db_dir = os.path.dirname(db_path)
        if db_dir and not os.path.exists(db_dir):
            os.makedirs(db_dir)
            print(f"Created database directory: {db_dir}")

        db.create_all()
        print("Database tables created/checked.")

        if not User.query.filter_by(username='bankuser').first():
            demo_user = User(username='bankuser', email='bank@example.com')
            db.session.add(demo_user)
            db.session.commit()
            print(f"Added demo user '{demo_user.username}'.")

            demo_account = Account(user_id=demo_user.id, balance=1000.00)
            db.session.add(demo_account)
            db.session.commit()
            print(f"Added demo account for '{demo_user.username}' with initial balance {demo_account.balance:.2f}.")

    # --- ngrok Tunnel Setup ---
    authtoken = os.environ.get('NGROK_TOKEN')

    if authtoken:
        conf.get_default().auth_token = authtoken
        print("ngrok authtoken set.")
    else:
        print("NGROK_TOKEN environment variable not set. Running Flask locally without ngrok tunnel.")

    if authtoken:
        try:
            global public_url
            public_url = ngrok.connect(5000).public_url
            print(f"\nngrok tunnel started at: {public_url}\n")
            print("Access your application via the ngrok URL shown above (e.g., in Google Colab).")
        except Exception as e:
            print(f"Failed to start ngrok tunnel: {e}")
            print("Running Flask locally without ngrok tunnel. Access at http://127.0.0.1:5000")
            public_url = "http://127.0.0.1:5000"

    app.run(host="0.0.0.0", port=5000, use_reloader=False, debug=True)

DeepSeek API client initialized successfully using userdata.
Database tables created/checked.
ngrok authtoken set.

ngrok tunnel started at: https://d1f8-34-169-67-127.ngrok-free.app

Access your application via the ngrok URL shown above (e.g., in Google Colab).
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 03:42:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 03:42:37] "GET /api/status HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 03:42:37] "GET /login?next=/api/status HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 03:42:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 03:42:48] "POST /login HTTP/1.1" 200 -
<ipython-input-4-5c396c818ec5>:56: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)

## bank solution


In [ ]:
# --- Part 1: Library Installations ---
!pip install Flask Flask-CORS Flask-SQLAlchemy pyngrok colab-env Flask-Login openai -q
!pip install crewai -q
!pip install 'crewai[tools]' -q

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from openai import OpenAI
from pydantic import Field
import colab_env

In [3]:
# --- Part 2: Import Libraries ---
# Import modules and classes needed for the application.

# Standard Library Imports
import os # Provides a way of using operating system dependent functionality (like environment variables)
import random # Used for adding jitter to retry delays
import time # Used for pausing execution (sleep) in retry logic
import traceback # Used for printing detailed error information (stack traces)

# Third-Party Library Imports
from flask import Flask, jsonify, request, send_from_directory, render_template, redirect, url_for, flash # Core Flask components
from flask_cors import CORS # Flask extension for handling Cross-Origin Resource Sharing
from flask_sqlalchemy import SQLAlchemy # Flask extension for SQLAlchemy (database interaction)
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user # Flask extension for user session management
from pyngrok import ngrok, conf # Used for creating a public tunnel to the local Flask server
from sqlalchemy.orm import relationship # SQLAlchemy component for defining relationships between models

# LLM Integration Imports (Using OpenAI client which is compatible with DeepSeek)
from openai import OpenAI, APIStatusError # OpenAI client for interacting with the LLM API, and a specific error type

# Colab Environment Imports (Optional, for setting env vars securely in Colab)
# This is specific to Google Colab. userdata is used for securely accessing secrets.
# The get_ipython() check helps make it compatible outside of Colab as well.
try:
    from google.colab import userdata # Securely access secrets stored in Colab
    from IPython import get_ipython # Check if running in IPython/Colab environment
except ImportError:
    # Provide dummy objects if not in Colab to avoid errors
    class DummyUserData:
        def get(self, key, default=None):
            return default
    userdata = DummyUserData()
    def get_ipython():
        return None
    print("Not running in Google Colab, google.colab.userdata not available.")


# CrewAI Imports
from crewai import Agent, Task, Crew, Process # Core CrewAI components for defining agents, tasks, and crews
from crewai.tools import BaseTool # Base class for creating custom tools that agents can use


# --- Part 3: Flask App Initialization and Configuration ---
# Set up the core Flask application instance and its basic configuration.

# Create the Flask application instance.
# static_folder and template_folder are set to common paths in Colab environment.
# Adjust these paths if your static and template files are located elsewhere.
app = Flask(__name__, static_folder='/content/static', template_folder='/content/templates')
CORS(app) # Enable CORS for the application, allowing requests from any origin

# Set a secret key. Gets from environment variable 'SECRET_KEY' first, falls back to a default.
# CHANGE THIS DEFAULT KEY IN PRODUCTION. It's crucial for security (e.g., signing session cookies).
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY', 'a_very_insecure_default_key_please_change_me_12345')

# --- Part 4: Database Configuration and Initialization ---
# Set up the database connection using Flask-SQLAlchemy.

# Configure SQLAlchemy to use an SQLite database file located at /content/site.db
# This path is suitable for Colab. Change it for other environments (e.g., 'sqlite:///site.db' for current dir)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/site.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # Disable SQLAlchemy modification tracking (saves overhead)

db = SQLAlchemy(app) # Initialize the SQLAlchemy extension with the Flask app


# --- Part 5: Flask-Login Setup ---
# Set up the Flask-Login extension for managing user sessions.

login_manager = LoginManager() # Create a LoginManager instance
login_manager.init_app(app) # Initialize Flask-Login with the Flask app
login_manager.login_view = 'login' # Set the name of the login route (function name). Flask-Login will redirect here if @login_required fails.


# --- Part 6: Database Models (SQLAlchemy ORM) ---
# Define Python classes that map to database tables, using SQLAlchemy's ORM.
# User model integrates with Flask-Login via UserMixin.

class User(db.Model, UserMixin): # UserMixin adds properties and methods required by Flask-Login (is_authenticated, get_id, etc.)
    # Define columns for the 'user' table
    __tablename__ = 'user' # Explicit table name
    id = db.Column(db.Integer, primary_key=True) # Primary key, auto-incrementing integer
    username = db.Column(db.String(80), unique=True, nullable=False) # Username, must be unique and not null
    email = db.Column(db.String(120), unique=True, nullable=False) # Email, must be unique and not null
    # NOTE: Password field is missing for simplicity. In a real app, add a password_hash field.
    # password_hash = db.Column(db.String(128), nullable=False) # Store hashed password

    # Define a relationship to the Account model.
    # 'account' attribute on a User object will hold the associated Account object.
    # 'backref' creates an 'owner' attribute on the Account model to access the related User object.
    # 'uselist=False' indicates a one-to-one relationship (each user has at most one account).
    account = relationship('Account', backref='owner', uselist=False)

    def __repr__(self):
        # String representation for debugging, shows username
        return f'<User {self.username}>'

class Account(db.Model):
    # Define columns for the 'account' table
    __tablename__ = 'account' # Explicit table name
    id = db.Column(db.Integer, primary_key=True) # Primary key
    # Foreign key linking to the 'user' table's id column.
    # 'unique=True' enforces that each user_id can only appear once in this table (one-to-one).
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), unique=True, nullable=False)
    balance = db.Column(db.Float, nullable=False, default=0.0) # Account balance, defaults to 0.0

    # 'owner' attribute on an Account object will hold the related User object due to backref

    def __repr__(self):
        # String representation for debugging, shows user ID and balance
        return f'<Account User:{self.user_id} Balance:{self.balance:.2f}>'


# --- Part 7: Flask-Login User Loader ---
# Callback function required by Flask-Login to load a user from the database.

@login_manager.user_loader
def load_user(user_id):
    """
    Loads a user from the database given their ID stored in the session by Flask-Login.
    Returns the User object or None if the user ID is invalid or not found.
    """
    if user_id is not None:
        try:
            # Flask-Login provides user_id as a string, convert to int for database query.
            # Query the User model by its primary key.
            return User.query.get(int(user_id))
        except (ValueError, TypeError):
            # Handle cases where user_id from session is not a valid integer or is None unexpectedly
            print(f"Warning: Invalid user_id found in session: {user_id}")
            return None # Return None if user ID is invalid
    return None # Return None if user_id is None (e.g., session cookie is empty or expired)


# --- Part 8: Global Variable for ngrok URL ---
# Initialize a variable to hold the public ngrok URL.

# Initialize with the default local Flask address. Will be updated if ngrok starts.
public_url = "http://127.0.0.1:5000"


# --- Part 9: DeepSeek API Client Setup ---
# Initialize the LLM client, checking for the API key securely.

# Retrieve DeepSeek API key securely from Colab userdata if in Colab, otherwise from environment variables.
# This method is specific to Google Colab. In other environments, use os.environ.get('DEEPSEEK_API_KEY').
# get_ipython() is used to check if the code is running in an IPython environment (like a Colab notebook).
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY') if 'google.colab' in str(get_ipython()) else os.environ.get('DEEPSEEK_API_KEY')


client = None # Initialize LLM client globally to None
if deepseek_api_key:
    try:
        # Initialize the OpenAI client. DeepSeek's API is compatible with the OpenAI client library.
        # The base_url is set to the DeepSeek API endpoint.
        client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
        print("DeepSeek API client initialized successfully.")
    except Exception as e:
        # Catch any errors during client initialization (e.g., invalid key format)
        print(f"ERROR: Failed to initialize DeepSeek API client: {e}")
        client = None # Ensure client is None if initialization fails
else:
    # Warning if the API key is not found
    print("WARNING: DEEPSEEK_API_KEY not found in secrets/environment variables. LLM features and agent functionality will be limited or unavailable.")


# --- Part 10: General LLM Call with Retry Logic ---
# A utility function to call the LLM, handling transient errors.

def call_llm_with_retry(prompt, max_retries=3):
    """
    Calls the LLM (DeepSeek via OpenAI client) with built-in retry logic.
    Handles common retriable API errors (rate limits, server errors) with exponential backoff and jitter.
    This function is suitable for general chat queries or tasks that don't require specific tools.
    Agents, when deciding which tool to use or generating thoughts, implicitly use the client
    assigned to them; this function is more for direct calls.
    Returns the LLM response string or an informative error message string if calls fail after retries.
    """
    if client is None:
        # If client wasn't initialized due to missing API key, return error
        return "Error: LLM client is not initialized. Cannot process request."

    # Define a system instruction to guide the LLM's persona and scope within the SuperApp context.
    system_instruction = (
        "You are a helpful and friendly AI assistant for a SuperApp that includes banking, "
        "shopping, and travel features. Your primary goal is to assist the user within these domains. "
        "Remember, this is FIRST a BANKING application. "
        "You are also being developed for flight planning optimization, drawing on principles "
        "from Newton, Galileo, Einstein, and Hinton. Keep responses concise and relevant."
    )

    messages = [
        {"role": "system", "content": system_instruction}, # System message sets the persona
        {"role": "user", "content": prompt}, # User message with the actual query
    ]

    # Loop up to max_retries to attempt the API call
    for i in range(max_retries):
        try:
            # Make the chat completion call using the initialized client
            response = client.chat.completions.create(
                model="deepseek-reasoner", # Specify the DeepSeek model name to use
                max_tokens=2024, # Limit the maximum number of tokens in the response
                temperature=0.7, # Controls randomness (0.0 is deterministic, higher is more creative)
                top_p=1, # Controls diversity of tokens (higher value includes more diverse tokens)
                messages=messages, # The conversation history/prompt list
                stream=False # Get the full response at once (True would stream tokens)
            )
            # Check the response structure and extract the content
            if response and response.choices and response.choices[0] and response.choices[0].message:
                return response.choices[0].message.content # Return the successful LLM's text response
            else:
                # Handle cases where the API call succeeds but the response structure is unexpected or empty.
                print(f"Warning: LLM returned empty or unexpected response structure (Attempt {i+1}/{max_retries})")
                if i < max_retries - 1:
                    # Consider a small delay even for structural issues before retrying
                     time.sleep(1)
                     continue # Try again with the next loop iteration
                else:
                     # If max retries reached and response is still empty/unexpected
                     return "Error: LLM returned an empty or unexpected response structure after retries."


        except APIStatusError as e:
            # Handle API errors (e.g., 4xx or 5xx status codes from the API)
            print(f"DeepSeek APIStatusError ({e.status_code}): {e.response} (Attempt {i+1}/{max_retries})")
            # Check for specific status codes that are often temporary and worth retrying (rate limits, server errors)
            if e.status_code == 429 or e.status_code >= 500: # 429 Too Many Requests, 5xx Server Error
                if i < max_retries - 1: # If there are retries left (not the last attempt)
                    # Implement exponential backoff with jitter: wait 2^i seconds + a random fraction.
                    delay = (2 ** i) + random.random()
                    print(f"Retrying in {delay:.2f} seconds...")
                    time.sleep(delay)
                    # Continue to the next iteration of the loop to retry the API call
                    continue
                else:
                    # Max retries reached for a retriable error.
                    print(f"Max retries reached for APIStatusError.")
                    traceback.print_exc() # Print detailed error stack for debugging
                    # Return an error message indicating max retries were exhausted
                    return f"Error from AI Agent service ({e.status_code}): {e.message}. Max retries exhausted."
            else:
                # Non-retriable error (e.g., 401 Unauthorized, 404 Not Found, 400 Bad Request).
                # These errors are unlikely to resolve with retries, so print error and return message immediately.
                 print(f"Non-retriable APIStatusError: ({e.status_code}): {e.response}")
                 traceback.print_exc()
                 return f"Error from AI Agent service ({e.status_code}): {e.message}"
        except Exception as e:
            # Handle any other unexpected errors that might occur during the API call or response processing
            print(f"An unexpected error occurred during LLM call (Attempt {i+1}/{max_retries}): {e}")
            traceback.print_exc() # Print detailed traceback for unexpected errors
            if i < max_retries - 1:
                 # Simple delay for unexpected errors before retrying
                 time.sleep(1)
                 continue # Continue to the next iteration
            else:
                 # If max retries reached for an unexpected error
                 return f"An unexpected error occurred with the AI Agent after retries: {str(e)}. Please try again later."

    # This line should only be reached if the loop finishes without returning
    # or raising an exception, which implies all retries failed to produce a valid response.
    return "Failed to get response from LLM after multiple retries."


# --- Part 11: Functions for Banking Operations (Used by CrewAI Tools) ---
# These functions encapsulate the core business logic for interacting with the database models.
# They are called by the CrewAI Tool classes.
# They MUST be called within a Flask application context (`with app.app_context():`)
# when executed by a CrewAI agent/tool, as agent execution often happens in a
# different thread or context than the original Flask request.

def get_user_balance(user_id: int):
    """
    Fetches the current balance for a given user ID from the database.
    Requires Flask app context to access db and models.
    Returns the balance (float) or None if user/account not found.
    """
    # Assume this function is always called from within a 'with app.app_context():' block
    user = User.query.get(user_id) # Query the User model by primary key
    if user and user.account: # Check if user exists and has an associated account
        return user.account.balance # Return the balance
    return None # Indicate user or account was not found

def perform_deposit(user_id: int, amount: float):
    """
    Adds funds to a user's account in the database.
    Requires Flask app context to access db and models.
    Includes validation for amount.
    Returns success status (bool) and new balance (float) or error message (str).
    """
    # Basic validation for the deposit amount
    if amount is None or not isinstance(amount, (int, float)) or float(amount) <= 0:
        return False, "Invalid deposit amount. Must be a positive number."

    # Assume this function is always called from within a 'with app.app_context():' block
    user = User.query.get(user_id) # Get the user
    if user and user.account: # Check if user and account exist
        try:
            user.account.balance += float(amount) # Add the amount to the balance
            db.session.commit() # Save changes to the database transactionally
            return True, user.account.balance # Return success status and the new balance
        except Exception as e:
            # Catch any database errors during the commit process
            db.session.rollback() # Roll back the session to undo any partial changes
            print(f"Database error performing deposit for user {user_id}: {e}")
            return False, f"Database error during deposit: {str(e)}" # Return failure status and error message
    return False, "User or account not found." # Return failure if user or account doesn't exist

def perform_withdrawal(user_id: int, amount: float):
    """
    Removes funds from a user's account in the database.
    Requires Flask app context to access db and models.
    Includes validation for amount and check for sufficient funds.
    Returns success status (bool) and new balance (float) or error message (str).
    """
    # Basic validation for the withdrawal amount
    if amount is None or not isinstance(amount, (int, float)) or float(amount) <= 0:
        return False, "Invalid withdrawal amount. Must be a positive number."

    # Assume this function is always called from within a 'with app.app_context():' block
    user = User.query.get(user_id) # Get the user
    if user and user.account: # Check if user and account exist
        withdrawal_amount = float(amount)
        # Check for sufficient funds BEFORE attempting the database transaction
        if user.account.balance < withdrawal_amount:
            return False, "Insufficient funds." # Return failure due to business logic constraint

        try:
            user.account.balance -= withdrawal_amount # Subtract the amount
            db.session.commit() # Save changes to the database transactionally
            return True, user.account.balance # Return success status and the new balance
        except Exception as e:
            # Catch any database errors during the commit process
            db.session.rollback() # Roll back the session to undo any partial changes
            print(f"Database error performing withdrawal for user {user_id}: {e}")
            return False, f"Database error during withdrawal: {str(e)}" # Return failure status and error message
    return False, "User or account not found." # Return failure if user or account doesn't exist


# --- Part 12: Create CrewAI Tools from Banking Functions ---
# These classes inherit from BaseTool and wrap the banking logic functions.
# Agents use these tools by name and provide input to the _run method.

class GetBalanceTool(BaseTool):
    name: str = "Get User Account Balance" # Name used by the agent to reference the tool
    description: str = "Fetches the current balance for a user's bank account given their user ID. Use this tool when the user asks about their account balance. Input should be the user ID as a string (e.g., '123')." # Description helps the agent decide when and how to use the tool

    def _run(self, user_id_str: str = None) -> str:
        """
        Executes the get_user_balance function.
        Called by the CrewAI agent. Receives input from the agent as a string.
        Must return a string.
        """
        # Validate and convert the input provided by the agent
        if user_id_str is None or not user_id_str.strip():
             # Agent failed to provide input despite description
             return "Error: User ID input is required for this tool."
        try:
            user_id = int(user_id_str.strip()) # Convert input string to integer
            if user_id <= 0:
                 return "Error: User ID must be a positive integer."
        except ValueError:
            # Input was provided but not a valid integer string
            return "Error: Invalid User ID format provided. Must be an integer string (e.g., '123')."

        # Crucially, call the underlying banking function within the Flask app context.
        # CrewAI runs tool execution potentially in a background thread/process where
        # the main Flask app context is not automatically available.
        with app.app_context():
             balance = get_user_balance(user_id) # Call the banking logic function

        if balance is not None:
            # Return success message with formatted balance as a string
            return f"Current balance: {balance:.2f}"
        else:
            # Return error if user/account not found by the banking function
            return f"Error: Could not find account for user ID {user_id} or account is missing."

class PerformDepositTool(BaseTool):
    name: str = "Perform Bank Deposit" # Name used by the agent
    description: str = "Deposits a specified amount into a user's bank account. Use this tool when the user explicitly asks to deposit money. The tool requires input in the format 'user_id,amount' (e.g., '123,50.00'). Ensure user_id is an integer and amount is a positive number. Both parts are required, separated by a comma." # Description guiding the agent's usage and input format

    def _run(self, tool_input: str = None) -> str:
        """
        Executes the perform_deposit function.
        Called by the CrewAI agent. Receives input as a string in 'user_id,amount' format.
        Must return a string.
        """
        if tool_input is None or not tool_input.strip():
             return "Error: Input in format 'user_id,amount' is required."
        try:
            # Split input string into user_id and amount, handle potential extra commas
            # Use split(',', 1) to ensure only the first comma is used as a separator.
            parts = tool_input.split(',', 1)
            if len(parts) != 2:
                 # Input string doesn't contain exactly one comma separating two parts
                 return "Error: Invalid input format. Expected 'user_id,amount'."
            user_id_str, amount_str = [p.strip() for p in parts] # Strip whitespace from both parts
            user_id = int(user_id_str) # Convert user ID part to integer
            amount = float(amount_str) # Convert amount part to float
        except ValueError:
            # Handle cases where conversion to int or float fails
            return "Error: Invalid number format in input. Expected 'user_id,amount' with valid numbers."
        except IndexError:
             # Should be caught by len(parts) != 2 check, but included for robustness.
             return "Error: Input format incorrect. Expected 'user_id,amount'."


        # Perform basic validation on the parsed numerical inputs
        if user_id <= 0 or amount <= 0:
             return "Error: User ID must be positive, and deposit amount must be positive."

        # Call the underlying banking function within the Flask app context
        with app.app_context():
            success, result = perform_deposit(user_id, amount) # Call the banking logic function

        if success:
            # If the deposit function returned success=True, format and return the new balance
            return f"Deposit successful. New balance: {result:.2f}"
        else:
            # If the deposit function returned success=False, return the error message provided by it
            return f"Deposit failed: {result}"

class PerformWithdrawalTool(BaseTool):
    name: str = "Perform Bank Withdrawal" # Name used by the agent
    description: str = "Withdraws a specified amount from a user's bank account. Use this tool when the user explicitly asks to withdraw money. The tool requires input in the format 'user_id,amount' (e.g., '123,25.00'). Ensure user_id is an integer and amount is a positive number. Both parts are required, separated by a comma." # Description guiding agent usage

    def _run(self, tool_input: str = None) -> str:
        """
        Executes the perform_withdrawal function.
        Called by the CrewAI agent. Receives input as a string in 'user_id,amount' format.
        Must return a string.
        """
        if tool_input is None or not tool_input.strip():
             return "Error: Input in format 'user_id,amount' is required."
        try:
            # Split input string into user_id and amount
            parts = tool_input.split(',', 1)
            if len(parts) != 2:
                 return "Error: Invalid input format. Expected 'user_id,amount'."
            user_id_str, amount_str = [p.strip() for p in parts] # Strip whitespace
            user_id = int(user_id_str) # Convert user ID to integer
            amount = float(amount_str) # Convert amount to float
        except ValueError:
            # Handle cases where conversion fails
            return "Error: Invalid number format in input. Expected 'user_id,amount' with valid numbers."
        except IndexError:
             # Should be caught by len(parts) != 2 check
             return "Error: Input format incorrect. Expected 'user_id,amount'."

        # Perform basic validation on parsed inputs
        if user_id <= 0 or amount <= 0:
             return "Error: User ID must be positive, and withdrawal amount must be positive."

        # Call the underlying banking function within the Flask app context
        with app.app_context():
            success, result = perform_withdrawal(user_id, amount) # Call the banking logic function

        if success:
            # If the withdrawal function returned success=True, format and return the new balance
            return f"Withdrawal successful. New balance: {result:.2f}"
        else:
            # If the withdrawal function returned success=False, return the error message (e.g., "Insufficient funds.")
            return f"Withdrawal failed: {result}"


# --- Part 13: CrewAI Agent Definitions ---
# Define the agents with their roles, goals, backstories, LLM, and assigned tools.

banking_agent = Agent(
    role='Expert Banking Assistant', # The agent's role
    goal='Provide accurate information about user account status and handle basic banking operations (deposit, withdrawal) by using the provided banking tools. Respond ONLY to banking-related queries for the given user ID.', # The agent's overall objective
    backstory='You are a helpful, secure, and accurate AI embedded in a SuperApp banking feature. You are authorized to access user account data and perform transactions ONLY through your specialized banking tools. You understand instructions about user IDs and amounts and prioritize the user\'s financial security.', # Context about the agent
    verbose=True, # Set to True to see the agent's detailed thought process (steps, tool usage, outputs) in the console logs
    llm=client, # Assign the initialized LLM client to this agent
    tools=[GetBalanceTool(), PerformDepositTool(), PerformWithdrawalTool()] # Provide the banking tools this agent can use
)

# Agent for the travel domain within the SuperApp. It does NOT have banking tools.
travel_agent = Agent(
    role='Experienced Travel Planner', # The agent's role
    goal='Suggest travel options based on user preferences and answer travel-related questions.', # Goal for the travel agent
    backstory='You are an AI assistant specializing in finding the best flights and travel deals for the SuperApp users. You do not have access to banking tools or information and should decline banking requests.', # Backstory for the travel agent
    verbose=True, # Set to True for verbose logging of this agent's process
    llm=client, # Assign the initialized LLM client to this agent
    # Add travel-specific tools here when implemented (e.g., FlightSearchTool, HotelBookingTool)
    tools=[] # This agent does not have access to banking tools
)


# --- Part 14: CrewAI Task Definitions ---
# Define the tasks that agents will perform. Task descriptions guide the agent's actions.

# Task for the banking agent. It is designed to interpret the user's query,
# understand the user ID context provided in the description, and guide the agent
# to use the correct banking tool with the properly formatted input.
banking_task = Task(
    description=(
        "Analyze the user's banking query: '{user_query}'. "
        "The user's ID is {user_id}. " # Inject user ID into the task description for agent context
        "Based on the query, decide which banking tool to use. You MUST use one of your provided tools if the query is a banking request."
        "Be extremely precise in using the tool and providing its input EXACTLY as required by the tool description."
        "- If the user asks for their account balance, use the 'Get User Account Balance' tool. The required input is the user ID. Provide ONLY '{user_id}' as input to the tool."
        "- If the user asks to deposit money, use the 'Perform Bank Deposit' tool. Carefully extract the exact numerical amount from the query (e.g., '50 dollars', '€100'). Provide input to the tool in the format 'user_id,amount'. For example, if the user says 'deposit 50 dollars', the input should be '{user_id},50.00'. Ensure the amount is a positive number."
        "- If the user asks to withdraw money, use the 'Perform Bank Withdrawal' tool. Carefully extract the exact numerical amount from the query (e.g., '25 euros', '$75'). Provide input to the tool in the format 'user_id,amount'. For example, if the user says 'withdraw 25 euros', the input should be '{user_id},25.00'. Ensure the amount is a positive number."
        "If the user's query is ambiguous, unclear, about non-banking topics for this specific user ID, or you cannot extract a clear positive amount for a transaction, state clearly and politely that you can only assist with clear banking requests for the logged-in user and their account, and explain what information you need (e.g., 'Please specify the exact numerical amount you wish to deposit or withdraw'). Do NOT attempt a transaction if the amount is unclear or zero/negative."
        "Your final answer MUST be a clear and concise response based on the outcome of the tool execution (the string output returned by the tool, e.g., 'Deposit successful. New balance: 1050.00') or your analysis if no tool was used."
        "\n\nUser Query: {user_query}" # Reiterate the user's query for the agent at the end
    ),
    expected_output='A clear and concise banking response, including the result of any performed transaction or balance inquiry, or a message indicating inability to process a non-banking or unclear request.', # Describes the expected format of the agent's final answer
    agent=banking_agent, # Assign the task to the banking agent
    # Context can be added here if this task depends on output of other tasks (not used in this simple sequential crew)
)

# Task for the travel agent.
travel_planning_task = Task(
    description="Analyze the user's query: '{user_query}'. Determine if it is a travel-related request. If it is, provide helpful travel suggestions or answer the travel-related question. If it is not a travel query, state clearly and politely that you can only assist with travel planning.", # Description for the travel agent's task
    expected_output='Relevant travel options, advice, information, or a statement indicating the query is not travel-related.', # Expected output format
    agent=travel_agent, # Assign the task to the travel agent
)


# --- Part 15: Flask Routes ---
# Define the URL endpoints (routes) that the Flask application will respond to.

@app.route('/')
def index():
    """Renders the main index page."""
    # Assumes an index.html template exists in the templates folder /content/templates
    # Pass the ngrok public URL to the template so the frontend can use the correct API endpoint base URL.
    # This is useful for linking to API endpoints from static HTML in the frontend.
    return render_template('index.html', public_url=public_url)

@app.route('/favicon.ico')
def favicon():
    """Serves the favicon from the static folder."""
    # Assumes favicon.ico is in the static folder /content/static
    return send_from_directory(app.static_folder, 'favicon.ico', mimetype='image/vnd.microsoft.icon')

DeepSeek API client initialized successfully.








In [ ]:
# --- Part 16: Original API Endpoints for Banking ---
# These endpoints provide a direct, non-agent way to interact with banking operations.
# They are protected by Flask-Login's @login_required decorator, ensuring only logged-in users can access them.

@app.route('/api/status', methods=['GET'])
@login_required # Requires user to be logged in to access this endpoint
def get_status():
    """API endpoint to get the current user's account balance."""
    # current_user object is available because the @login_required decorator ensures the user is logged in and loaded
    user_account = current_user.account
    if not user_account:
        # This case shouldn't happen if registration always creates an account, but it's a safeguard
        # User is logged in but has no account associated.
        return jsonify(isLoggedIn=True, username=current_user.username, balance=None, message="No account found for user"), 404 # Not Found

    # Return user status and balance as JSON response
    return jsonify(
        isLoggedIn=True,
        username=current_user.username,
        balance=user_account.balance,
        message="User status and balance retrieved."
    ), 200 # OK status code

@app.route('/api/deposit', methods=['POST'])
@login_required # Requires user to be logged in to perform a deposit
def deposit():
    """API endpoint to perform a deposit."""
    data = request.json # Get data from the request body (expected JSON payload from frontend)
    amount = data.get('amount') # Extract the 'amount' key from the JSON data

    # Basic validation of the deposit amount
    if amount is None or not isinstance(amount, (int, float)) or float(amount) <= 0:
        return jsonify(success=False, message="Invalid deposit amount. Must be a positive number."), 400 # Bad Request

    # Get the logged-in user's account from the current_user object
    user_account = current_user.account
    if not user_account:
        return jsonify(success=False, message="No account found for user."), 404 # Not Found

    try:
        # Perform the deposit by adding the amount to the balance
        user_account.balance += float(amount)
        db.session.commit() # Save changes to the database
        # Return success response with the new balance
        return jsonify(success=True, newBalance=user_account.balance, message=f"Deposited {amount:.2f}. New balance: {user_account.balance:.2f}"), 200 # OK

    except Exception as e:
        # Handle any database errors that might occur during the commit
        db.session.rollback() # Rollback the session to undo changes in case of error
        print(f"Error processing deposit via direct API: {e}")
        traceback.print_exc() # Print detailed traceback to logs for debugging
        return jsonify(success=False, message=f"Error processing deposit: {str(e)}"), 500 # Internal Server Error

@app.route('/api/withdraw', methods=['POST'])
@login_required # Requires user to be logged in to perform a withdrawal
def withdraw():
    """API endpoint to perform a withdrawal."""
    data = request.json # Get data from the request body (expected JSON payload)
    amount = data.get('amount') # Extract the 'amount' key from the JSON data

    # Basic validation of the withdrawal amount
    if amount is None or not isinstance(amount, (int, float)) or float(amount) <= 0:
        return jsonify(success=False, message="Invalid withdrawal amount. Must be a positive number."), 400 # Bad Request

    # Get the logged-in user's account
    user_account = current_user.account
    if not user_account:
        return jsonify(success=False, message="No account found for user."), 404 # Not Found

    withdrawal_amount = float(amount)
    # Check for sufficient funds *before* attempting the withdrawal transaction
    if user_account.balance < withdrawal_amount:
        return jsonify(success=False, message="Insufficient funds."), 400 # Bad Request (due to insufficient funds)

    try:
        # Perform the withdrawal by subtracting the amount
        user_account.balance -= withdrawal_amount
        db.session.commit() # Save changes to the database
        # Return success response with the new balance
        return jsonify(success=True, newBalance=user_account.balance, message=f"Withdrew {withdrawal_amount:.2f}. New balance: {user_account.balance:.2f}"), 200 # OK

    except Exception as e:
        # Handle any database errors that might occur during the commit
        db.session.rollback() # Rollback the session to undo changes in case of error
        print(f"Error processing withdrawal via direct API: {e}")
        traceback.print_exc() # Print detailed traceback to logs for debugging
        return jsonify(success=False, message=f"Error processing withdrawal: {str(e)}"), 500 # Internal Server Error

# --- Part 17: Agent Chat Endpoint (Uses the general LLM call, NOT the CrewAI agents/tools) ---
# This endpoint provides a simple chat interface using the LLM directly for general queries.
# For queries requiring actions (like banking transactions), the /api/run_crew
# endpoint is the intended interface for multi-agent processing with tools.

@app.route('/api/agent_chat', methods=['POST'])
@login_required # Requires user to be logged in to interact with agents that handle user data
def agent_chat():
    """
    Integrates with DeepSeek LLM via OpenAI client for general chat responses.
    Uses the simple call_llm_with_retry function.
    Consider deprecating or routing this to /api/run_crew if all AI interaction
    should go through agents for consistency.
    """
    data = request.json # Get data from the request body
    user_message = data.get('message', '') # Extract the user's message

    if not user_message:
        return jsonify(success=False, message="No message provided for agent chat."), 400 # Bad Request

    # Use the general LLM call function defined earlier
    try:
        agent_response = call_llm_with_retry(prompt=user_message)

        # Check the response content for indicators of internal errors from the LLM function
        if "Error:" in agent_response or "WARNING:" in agent_response:
             # Depending on desired behavior, you might return an error status here
             # or just the message content which explains the issue.
             status_code = 500 if "Error:" in agent_response else 200
             return jsonify(success=True, response=agent_response, message="Note: Issues encountered during AI processing."), status_code
        else:
             return jsonify(success=True, response=agent_response), 200 # OK

    except Exception as e: # Catch any unexpected errors not handled within call_llm_with_retry
        print(f"Unhandled error in /api/agent_chat route: {e}")
        traceback.print_exc() # Print detailed traceback to logs
        return jsonify(success=False, message=f"An unexpected error occurred with the AI service: {str(e)}. Please try again."), 500 # Internal Server Error


# --- Part 18: New CrewAI Endpoint ---
# This is the primary endpoint for multi-agent interactions that can use tools.

@app.route('/api/run_crew', methods=['POST'])
@login_required # Requires user to be logged in to interact with agents that handle user data
def run_crew():
    """
    API endpoint to trigger a CrewAI process based on user query and crew type.
    Handles routing the query to the appropriate agent crew (banking or travel).
    """
    data = request.json # Get data from the request body
    user_query = data.get('query', '') # Extract the user's query
    # Default to 'banking' if no crew_type is specified, make it lowercase for consistent checking
    crew_type = data.get('crew_type', 'banking').lower()

    if not user_query:
        return jsonify(success=False, message="No query provided for the agent crew."), 400 # Bad Request

    # Check if the LLM client is initialized before attempting to run a crew that requires it
    if client is None:
         return jsonify(success=False, message="AI Agent service is currently unavailable (LLM client not configured). Please ensure DEEPSEEK_API_KEY is set)."), 503 # Service Unavailable


    # Get the current user's ID. This is essential for the banking crew/tasks/tools
    # to know which user's account to operate on.
    # We pass this context into the task description for the banking agent.
    current_user_id = current_user.id


    # --- Crew Initialization and Execution ---
    try:
        result = "No crew executed." # Default result if crew_type is invalid or no process runs

        if crew_type == 'banking':
            print(f"Received banking query for user {current_user_id}: '{user_query}'")

            # Prepare the banking task description by dynamically formatting it with the user's query and ID.
            # We create a new Task instance here because task descriptions are immutable
            # and we need to inject dynamic user-specific data.
            formatted_task_description = banking_task.description.format(
                user_query=user_query,
                user_id=current_user_id
            )

            # Initialize the banking crew
            banking_crew = Crew(
                agents=[banking_agent], # The crew consists only of the banking agent for banking tasks
                tasks=[Task(description=formatted_task_description, # Use the dynamically formatted description
                            expected_output=banking_task.expected_output,
                            agent=banking_agent)], # Assign the task back to the banking agent
                process=Process.sequential, # Tasks executed in order (only one task here, but good practice)
                verbose=True, # Show detailed agent execution steps in logs (useful for debugging)
            )

            print("Starting banking CrewAI process...")
            # Execute the crew's tasks. The agent will use tools if needed based on the task description.
            result = banking_crew.kickoff()
            print("Banking CrewAI process finished.")

        elif crew_type == 'travel':
            print(f"Received travel query for user {current_user_id}: '{user_query}'")
            # Prepare the travel task description by dynamically formatting it with the user's query
            # Create a new Task instance with the formatted description
            formatted_task_description = travel_planning_task.description.format(user_query=user_query)

            # Initialize the travel crew
            travel_crew = Crew(
                agents=[travel_agent], # The crew consists only of the travel agent for travel tasks
                tasks=[Task(description=formatted_task_description, # Use the dynamically formatted description
                             expected_output=travel_planning_task.expected_output,
                             agent=travel_agent)], # Assign the task back to the travel agent
                process=Process.sequential, # Tasks executed in order (only one task here, but good practice)
                verbose=True, # Show detailed agent execution steps in logs (useful for debugging)
            )

            print("Starting travel CrewAI process...")
            # Execute the crew's tasks. The agent will use tools if needed based on the task description.
            result = travel_crew.kickoff()
            print("Travel CrewAI process finished.")

        else:
             # Handle invalid crew type request
             return jsonify(success=False, message=f"Invalid crew type specified: '{crew_type}'. Available types: 'banking', 'travel'."), 400 # Bad Request


        # Return the final result from the executed crew
        return jsonify(success=True, result=result), 200 # OK

    except Exception as e:
        # Catch any errors during the CrewAI setup or execution process
        print(f"Error running CrewAI for {crew_type} crew: {e}")
        traceback.print_exc() # Print detailed traceback to logs
        # Return an informative error message to the client
        return jsonify(success=False, message=f"An error occurred while running the {crew_type} agent crew: {str(e)}. Check server logs for details."), 500 # Internal Server Error


# --- Part 19: Authentication Routes (Flask-Login) ---
# These routes handle user registration, login, and logout using Flask-Login.
# Note: Passwords are NOT hashed in this example for simplicity.
# In a real application, ALWAYS hash passwords securely before storing them
# and use a proper password verification library (e.g., Flask-Bcrypt or Werkzeug.security).

@app.route('/register', methods=['POST'])
def register():
    """Handles user registration."""
    # Prevent registration if already logged in
    if current_user.is_authenticated:
        return jsonify(message='Already logged in. Cannot register new account while logged in.', success=False), 400 # Bad Request

    # Get registration data from the form (assuming form-data or json with form-like structure)
    username = request.form.get('username')
    email = request.form.get('email')
    # Password would typically be required here and hashed before saving
    # password = request.form.get('password')

    # Basic validation for required fields (username and email in this simplified example)
    if not username or not email: # Simplified check without password
        return jsonify(message='Missing username or email.', success=False), 400 # Bad Request

    # Ensure database operations run within the application context
    # This is important because this route might be called outside of the
    # __main__ block's initial context setup if running with a web server.
    with app.app_context():
        # Check if username or email already exist in the database
        if User.query.filter_by(username=username).first():
            return jsonify(message='Username already exists.', success=False), 409 # 409 Conflict
        if User.query.filter_by(email=email).first():
            return jsonify(message='Email already exists.', success=False), 409 # 409 Conflict

        try:
            # Create new user object (add password hashing here in a real app)
            new_user = User(username=username, email=email)
            db.session.add(new_user) # Add the new user to the session
            db.session.commit() # Commit the session to save the user and get their database ID

            # Create a default account for the new user
            new_account = Account(user_id=new_user.id, balance=0.0)
            db.session.add(new_account) # Add the new account to the session
            db.session.commit() # Commit the session to save the account

            # Log the user in automatically after successful registration (optional but common)
            # login_user(new_user) # Uncomment this line if you want auto-login after register

            # Return success response
            return jsonify(message='Registration successful! You can now log in.', success=True), 201 # 201 Created

        except Exception as e:
            # Handle any database errors during the process
            db.session.rollback() # Rollback the session to undo changes on error
            print(f"Registration failed: {e}")
            traceback.print_exc() # Print detailed traceback to logs for debugging
            return jsonify(message=f'Registration failed: {str(e)}', success=False), 500 # Internal Server Error


@app.route('/login', methods=['GET', 'POST'])
def login():
    """Handles user login."""
    # Prevent login if already logged in
    if current_user.is_authenticated:
        # If already logged in, just confirm and return success
        return jsonify(message=f'Already logged in as {current_user.username}!', success=True), 200 # OK

    # Handle POST requests (login attempt)
    if request.method == 'POST':
        username = request.form.get('username')
        # Password would be needed here in a real app for verification against the stored hash
        # password = request.form.get('password')

        # Basic validation for required fields (username in this simplified example)
        if not username: # Simplified check without password
             return jsonify(message='Missing username.', success=False), 400 # Bad Request

        # Ensure database operations run within the application context
        with app.app_context():
            # Find the user by username
            user = User.query.filter_by(username=username).first()

            # In a real app, you'd verify the password hash here using the extracted password
            # Example with Flask-Bcrypt: if user and bcrypt.check_password_hash(user.password_hash, password):

            # Simplified check based on username existence only (!! NOT secure for production !!)
            if user:
                # If user is found (and password verified in a real app), log them in using Flask-Login
                login_user(user) # This sets the user ID in the session
                # Return success response. You might return user details or a token here too.
                return jsonify(message=f'Logged in as {user.username}!', success=True), 200 # OK
            else:
                 # User not found or credentials don't match (in real app)
                 # Provide a generic error message for security (avoiding "username not found" vs "wrong password")
                 return jsonify(message='Invalid username or password.', success=False), 401 # 401 Unauthorized

    # Handle GET requests to /login endpoint
    # A GET request to the login URL typically serves the login page in a traditional web app.
    # In an API context, it might just indicate the endpoint exists and expects POST.
    return jsonify(message='Please use POST request with username and password to login.', success=False), 405 # 405 Method Not Allowed


@app.route('/logout')
@login_required # Requires user to be logged in to log out
def logout():
    """Handles user logout."""
    # current_user is available here because of @login_required
    logout_user() # Log the user out using Flask-Login (clears the user ID from the session)
    # Return success response
    return jsonify(message='You have been logged out.', success=True), 200 # OK


# --- Part 20: Application Startup ---
# This block runs only when the script is executed directly (e.g., `python your_script_name.py` or in a notebook cell)
if __name__ == '__main__':
    # Set up database and create tables/demo user within application context
    # Operations that interact with the database via SQLAlchemy (like create_all)
    # need to be performed within a Flask application context.
    with app.app_context():
        # Ensure the directory for the SQLite database file exists
        db_path = app.config['SQLALCHEMY_DATABASE_URI'].replace('sqlite:///', '')
        db_dir = os.path.dirname(db_path)
        if db_dir and not os.path.exists(db_dir):
            os.makedirs(db_dir)
            print(f"Created database directory: {db_dir}")

        # Create database tables based on the defined models (User, Account)
        # This will create tables if they don't already exist.
        db.create_all()
        print("Database tables created/checked.")

        # Add a demo user ('bankuser') and account if they don't already exist.
        # This is useful for easily testing the application in development/notebook environments.
        # In a real production app, user creation happens securely via the /register route.
        if not User.query.filter_by(username='bankuser').first():
            # Create demo user object (add password hashing here in a real app)
            demo_user = User(username='bankuser', email='bank@example.com')
            db.session.add(demo_user) # Add demo user to the session
            db.session.commit() # Commit to save the user and get their database ID
            print(f"Added demo user '{demo_user.username}'.")

            # Add a demo account for the demo user with an initial balance
            demo_account = Account(user_id=demo_user.id, balance=1000.00)
            db.session.add(demo_account) # Add demo account to the session
            db.session.commit() # Commit to save the account
            print(f"Added demo account for '{demo_user.username}' with initial balance {demo_account.balance:.2f}.")
        else:
             print("Demo user 'bankuser' already exists.")


    # --- ngrok Tunnel Setup ---
    # Get ngrok authentication token from environment variables (e.g., set in Google Colab secrets).
    # Ensure you have added your NGROK_TOKEN to Colab secrets if running there, or set it as an environment variable elsewhere.
    authtoken = os.environ.get('NGROK_TOKEN')

    if authtoken:
        conf.get_default().auth_token = authtoken
        print("ngrok authtoken set.")
        try:
            # Connect ngrok tunnel to the Flask default port (5000)
            # Use 'global' keyword to update the public_url variable outside this 'if' scope
            global public_url
            http_tunnel = ngrok.connect(5000)
            public_url = http_tunnel.public_url # Get the publicly accessible URL
            print(f"\nngrok tunnel started at: {public_url}\n")
            print("Access your application via the ngrok URL shown above.")
            print("If running in Colab, click the ngrok URL to open in browser.")
            # Optional: Print ngrok tunnel details (verbose output)
            # print(f"ngrok tunnel details: {http_tunnel}")
        except Exception as e:
            # Handle errors during ngrok setup (e.g., invalid token, ngrok service not running, network issues)
            print(f"Failed to start ngrok tunnel: {e}")
            traceback.print_exc() # Print detailed error stack for debugging
            print("Running Flask locally without ngrok tunnel. Access at http://127.0.0.1:5000")
            # Keep the default local URL if ngrok fails
            public_url = "http://127.0.0.1:5000"
    else:
        # If NGROK_TOKEN is not set, skip ngrok setup
        print("NGROK_TOKEN environment variable not set.")
        print("Running Flask locally without ngrok tunnel. Access at http://127.0.0.1:5000")
        # Keep the default local URL
        public_url = "http://127.0.0.1:5000"

    # --- Start the Flask Development Server ---
    # app.run starts the Flask web server.
    # host="0.0.0.0" makes the server accessible externally (necessary for ngrok or other external access).
    # port=5000 is the default Flask port.
    # use_reloader=False is important when running in notebooks or environments with external processes like ngrok,
    # as the reloader can cause the script to run multiple times.
    # debug=True provides helpful debugging information and enables Flask's debug mode.
    app.run(host="0.0.0.0", port=5000, use_reloader=False, debug=True)

Database tables created/checked.
Added demo user 'bankuser'.
Added demo account for 'bankuser' with initial balance 1000.00.
ngrok authtoken set.

ngrok tunnel started at: https://a727-34-16-220-70.ngrok-free.app

Access your application via the ngrok URL shown above.
If running in Colab, click the ngrok URL to open in browser.
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:03] "GET /api/status HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:03] "GET /login?next=/api/status HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:18] "POST /register HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 04:37:25] "POST /login HTTP/1.1" 200 -
<ipython-input-3-08087b1f2836>:127: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (